# Simulation of Drug Transport across a Virtual Skin Membrane

### Initialize UG4 (for 2D and standard algebra)

In [1]:
InitUG(2, AlgebraType("CPU", 1));
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... fail                 *


### Create Domain

In [2]:
requiredSubsets = {"LIP", "COR", "BOTTOM_SC", "TOP_SC"}
gridName = "skin2d-aniso.ugx"
numRefs = 2

In [3]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain skin2d-aniso.ugx ... done.
Performing integrity check on domain ... done.
Refining(2): 1 2 done.


### Create Approximation space

In [4]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [5]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          680 |           32 |          608 |           20 |           20 |
| (lev,    1)    |         2613 |          783 |         1752 |           39 |           39 |
| (lev,    2)    |        10241 |         4367 |         5720 |           77 |           77 |
| (lev,    0, g) |          680 |           32 |          608 |           20 |           20 |
| (lev,    1, g) |         2613 |      

## Create a convection-diffusion-equation
Model parameter

In [1]:
K={
    ["LIP"] = 1.0, ["COR"] = 1.0,
}

D={
     ["LIP"] = 1, ["COR"] = 0.01, 
}


* Initializing: paths... done, bridge... done, plugins... fail                 *


In [7]:
elemDisc ={}

elemDisc["COR"] = ConvectionDiffusion("c", "COR", "fv1")
elemDisc["COR"]:set_diffusion(K["COR"]*D["COR"])
elemDisc["COR"]:set_mass_scale(K["COR"])

elemDisc["LIP"] = ConvectionDiffusion("c", "LIP", "fv1")
elemDisc["LIP"]:set_diffusion(K["LIP"]*D["LIP"])
elemDisc["LIP"]:set_mass_scale(K["LIP"])

In [8]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "c", "TOP_SC")
dirichletBnd:add(0.0, "c", "BOTTOM_SC")

In [9]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["LIP"])
domainDisc:add(elemDisc["COR"])
domainDisc:add(dirichletBnd)

## Create  solver

In [10]:
solver=SuperLU()

## Solve transient problem

In [11]:
u = GridFunction(approxSpace)
u:set(0.0)

In [12]:
local startTime = 0.0
local endTime = 100.0
local dt=0.25
util.SolveLinearTimeProblem(u, domainDisc, solver, VTKOutput(), "vtk/skin",
                            "ImplEuler", 1, startTime, endTime, dt); 

SolveLinearTimeProblem, Linear Solver setup:
SuperLU
>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 0.25
++++++ Assembling Matrix/Rhs for step size 0.25
++++++ TIMESTEP 1 END   (current time: 0.25) ++++++
++++++ TIMESTEP 2 BEGIN (current time: 0.25) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 2 END   (current time: 0.5) ++++++
++++++ TIMESTEP 3 BEGIN (current time: 0.5) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 3 END   (current time: 0.75) ++++++
++++++ TIMESTEP 4 BEGIN (current time: 0.75) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 4 END   (current time: 1) ++++++
++++++ TIMESTEP 5 BEGIN (current time: 1) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 5 END   (current time: 1.25) ++++++
++++++ TIMESTEP 6 BEGIN (current time: 1.25) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 6 END   (current time: 1.5) ++++++
++++++ TIMESTEP 7 BEGIN (current time: 1.5) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 